In [1]:
# Run if using on google collab, change path if using on a copy
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
path = '/content/drive/MyDrive/[02] School/[01] University/[02] Bachelor\'s Year 2/[02] Spring Semester/[04] Machine Learning/Colab Notebooks/ML - LGI/Project/'

Mounted at /content/drive


In [5]:
# run if running locally
path = ''

In [3]:
#standard initial libraries
from math import ceil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency

#sk-learn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier;


#setting seaborn visual style in plt
sns.set()

# Integration & Exploration

In [6]:
#import the data
traindata: pd.DataFrame = pd.read_csv(path + 'project_data/train.csv').set_index('Userid').drop('Observations', axis=1).drop_duplicates()
testdata: pd.DataFrame = pd.read_csv(path + 'project_data/test.csv').set_index('Userid').drop(['Registered', 'Observations'], axis=1)

traindata = traindata[traindata['Registered'] == 'Yes']
traindata = traindata.drop('Registered', axis=1)

In [1]:
metricFeatures: list[str] = ['Application order','Previous qualification score','Entry score','Age at enrollment','N units credited 1st period',
       'N units taken 1st period', 'N scored units 1st period',
       'N units approved 1st period', 'Average grade 1st period',
       'N unscored units 1st period', 'N units credited 2nd period',
       'N units taken 2nd period', 'N scored units 2nd period',
       'N units approved 2nd period', 'Average grade 2nd period',
       'N unscored units 2nd period','Social Popularity']
categoricalFeatures: list[str] =  ['Application mode','Marital status','Course','Previous qualification','Nationality',"Mother's qualification",
       "Father's qualification","Mother's occupation","Father's occupation"]
boolFeatures: list[str] = ['Morning shift participation','Displaced','Special needs','Debtor','Regularized Fees','Gender_Male','External Funding','International']
target = 'Success'

In [ ]:
traindata.info()

In [ ]:
for variable in categoricalFeatures:
  print(traindata[variable].value_counts())

In [50]:
X = traindata.drop(['Success'], axis = 1)
y = traindata['Success']

In [ ]:
remlist = categorical_features[:]
remlist.remove('Course')

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 15, test_size = 0.2)

y_train.replace({'Succeeded': 2, 'Holding on': 1, 'Gave up': 0}, inplace = True)
X_train_filled = X_train.fillna(X_train[metric_features].median())
X_train_filled = X_train_filled.fillna(X_train_filled[categorical_features].mode().iloc[0])
X_train_filled.drop(remlist, axis = 1, inplace = True)
X_train_filled = pd.get_dummies(X_train_filled, drop_first = True)
# X_train_filled.drop(['N units taken 2nd period', 'N units credited 2nd period', 'N units approved 2nd period'], axis = 1, inplace = True)
scaler = MinMaxScaler().fit(X_train_filled)
X_train_scaled = scaler.transform(X_train_filled)
X_train = pd.DataFrame(X_train_scaled, columns = X_train_filled.columns, index = X_train_filled.index)

y_val.replace({'Succeeded': 2, 'Holding on': 1, 'Gave up': 0}, inplace = True)
X_val_filled = X_val.fillna(X_val[metric_features].median())
X_val_filled = X_val_filled.fillna(X_val_filled[categorical_features].mode().iloc[0])
X_val_filled.drop(remlist, axis = 1, inplace = True)
X_val_filled = pd.get_dummies(X_val_filled, drop_first = True)
# X_val_filled.drop(['N units taken 2nd period', 'N units credited 2nd period', 'N units approved 2nd period'], axis = 1, inplace = True)
X_val_scaled = scaler.transform(X_val_filled)
X_val = pd.DataFrame(X_val_scaled, columns = X_val_filled.columns, index = X_val_filled.index)

testdata_filled = testdata.fillna(testdata[metric_features].median())
testdata_filled = testdata_filled.fillna(testdata_filled[categorical_features].mode().iloc[0])
# testdata_filled.drop('Observations', axis = 1, inplace = True)
testdata_filled.drop(remlist, axis = 1, inplace = True)
testdata_filled = pd.get_dummies(testdata_filled, drop_first = True)
# testdata_filled.drop(['N units taken 2nd period', 'N units credited 2nd period', 'N units approved 2nd period'], axis = 1, inplace = True)
testdata_scaled = scaler.transform(testdata_filled)
testdata = pd.DataFrame(testdata_scaled, columns = testdata_filled.columns, index = testdata_filled.index)

In [19]:
y.replace({'Succeeded': 2, 'Holding on': 1, 'Gave up': 0}, inplace = True)
# X_filled = X.fillna(X[metric_features].median())
# X_filled = X_filled.fillna(X_filled[categorical_features].mode().iloc[0])
# X_filled.drop(remlist, axis = 1, inplace = True)
X_filled = pd.get_dummies(X, dummy_na = False, drop_first = False)
X_train = X_filled
# X_train_filled.drop(['N units taken 2nd period', 'N units credited 2nd period', 'N units approved 2nd period'], axis = 1, inplace = True)

In [20]:
# testdata_filled = testdata.fillna(testdata[metric_features].median())
# testdata_filled = testdata_filled.fillna(testdata_filled[categorical_features].mode().iloc[0])
# testdata_filled.drop(remlist, axis = 1, inplace = True)
testdata_filled = pd.get_dummies(testdata, dummy_na=False, drop_first = False)
testdata = testdata_filled
# testdata_filled.drop(['N units taken 2nd period', 'N units credited 2nd period', 'N units approved 2nd period'], axis = 1, inplace = True)

In [ ]:
columns_validation = set(X_train.columns) - set(X_val.columns)

for col in columns_validation:
    X_val[col] = 0

for column in X_val.columns:
    if column not in X_train.columns:
        X_val.drop(column, axis = 1, inplace = True)

X_val = X_val[X_train.columns]

In [21]:
columns_validation = set(X_train.columns) - set(testdata.columns)

for col in columns_validation:
  testdata[col] = 0

for column in testdata.columns:
  if column not in X_train.columns:
    testdata.drop(column, axis = 1, inplace = True)

testdata = testdata[X_train.columns]

In [ ]:
X_train.isna().sum()

In [ ]:
for i, col in enumerate(metric_features):
  plt.figure(i)
  sns.boxplot(x=col, data=X_train)

# Preprocessing

## Get Dummy variables

## Missing Values

In [ ]:
X_train_filled = X_train.fillna(X_train[metric_features].mean())

In [ ]:
X_train_filled = X_train_filled.fillna(X_train_filled[categorical_features].mode().iloc[0])

In [ ]:
for i, col in enumerate(metric_features):
  plt.figure(i)
  sns.boxplot(x=col, data=X_train_filled)

In [ ]:
X_train_filled.isna().sum()

# Feature Selection

In [ ]:
univariatecheck = X_train[metric_features].var()

In [ ]:
for i in univariatecheck:
  print(np.round(i,5))

In [ ]:
def cor_heatmap(cor):
    plt.figure(figsize=(12,10))
    sns.heatmap(data = cor, annot = True, cmap = plt.cm.Reds, fmt='.1')
    plt.show()

In [ ]:
cor_spearman = pd.concat([X_train[metric_features], y_train], axis=1).corr(method ='spearman')
cor_heatmap(cor_spearman)

In [ ]:
cor_kendall = pd.concat([X_train[metric_features], y_train], axis=1).corr(method ='kendall')
cor_heatmap(cor_kendall)

In [ ]:
anova = SelectKBest(f_classif, k=5)

In [ ]:
X_anova = anova.fit_transform(X_train_filled[metric_features], y_train)

In [ ]:
selected_features = pd.Series(anova.get_support(), index = X_train_filled[metric_features].columns)
selected_features

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Mother's qualification")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Father's qualification")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Marital status")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Course")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Previous qualification")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Mother's occupation")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Father's occupation")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[[s for s in X_train.columns if s.startswith("Nationality")]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
cat_spearman = pd.concat([X_train[["External Funding", "Application mode", "Morning shift participation", "Displaced", "Special needs", "Debtor", "Regularized Fees", "Gender_Male", "International"]], y_train], axis = 1).corr(method = 'spearman')
cor_heatmap(cat_spearman)

In [ ]:
ls = X_train.columns
for i in ls:
    print(i)

| Predictor                    | Variance | Spearman | Kendall  | ANOVA   | Solution |
|------------------------------|----------|----------|----------|---------|----------|
| Application order            | Keep     | Keep     | Keep     | Discard | Keep     |
| Previous qualification score | Keep     | Keep     | Keep     | Discard | Keep     |
| Entry score                  | Keep     | Keep     | Keep     | Discard | Keep     |
| Age at enrollment            | Keep     | Keep     | Keep     | Keep    | Keep     |
| N units credited 1st period  | Keep     | Keep     | Keep     | Discard | Keep     |
| N units taken 1st period     | Keep     | Keep     | Keep     | Discard | Keep     |
| N scored units 1st period    | Keep     | Keep     | Keep     | Discard | Keep     |
| N units approved 1st period  | Keep     | Keep     | Keep     | Keep    | Keep     |
| Average grade 1st period     | Keep     | Keep     | Keep     | Keep    | Keep     |
| N unscored units 1st period  | Keep     | Keep     | Keep     | Discard | Keep     |
| N units credited 2nd period  | Keep     | Discard  | Discard  | Discard | Discard  |
| N units taken 2nd period     | Keep     | Discard  | Discard  | Discard | Discard  |
| N scored units 2nd period    | Keep     | Keep     | Keep     | Discard | Keep     |
| N units approved 2nd period  | Keep     | Discard  | Discard? | Keep    | Test     |
| Average grade 2nd period     | Keep     | Discard? | Keep     | Keep    | Test     |
| N unscored units 2nd period  | Keep     | Keep     | Keep     | Discard | Keep     |
| Social Popularity            | Keep     | Keep     | Keep     | Discard | Keep     |

# Model

In [ ]:
parameter_space_lr = {
    'penalty': ["l2", "elasticnet", None],
    'solver': ["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"],
    'class_weight': [None, 'balanced'],
    'C' : [0.2, 0.5, 0.7, 1], # inverse regularization parameter
}

In [ ]:
#running this will give some warnings when convergence is not reached- run the code below to have them not show up
import warnings
warnings.filterwarnings('ignore')

# clf = GridSearchCV(lr, parameter_space_lr, scoring = 'f1', cv = 10,)
# clf.fit(X_train,y_train)

# Best parameter set
print('------------------------------------------------------------------------------------------------------------------------')
# print('Best parameters found:\n', clf.best_params_)
print('Best parameters found:\n', gsdtmicro.best_params_)
print('Best parameters found:\n', gsdtmacro.best_params_)
print('Best parameters found:\n', gsdtweighted.best_params_)
# print('Best parameters found:\n', gsmlpc.best_params_)
print('------------------------------------------------------------------------------------------------------------------------')


------------------------------------------------------------------------------------------------------------------------
Best parameters found:
 {'criterion': 'entropy', 'max_depth': 4, 'splitter': 'best'}
Best parameters found:
 {'criterion': 'gini', 'max_depth': 4, 'splitter': 'best'}
Best parameters found:
 {'criterion': 'gini', 'max_depth': 10, 'splitter': 'random'}
------------------------------------------------------------------------------------------------------------------------


In [22]:
model = RandomForestClassifier(800)
model.fit(X_train, y)

RandomForestClassifier(n_estimators=800)

In [23]:
test_pred = model.predict(testdata) #test preds

In [24]:
index = testdata.index.T
answer = pd.DataFrame([index, test_pred]).T
answer.columns = ['Userid','Success']
answer.replace({'Success': {0: 'Gave Up', 1: 'Holding on', 2: 'Succeeded'}}, inplace = True)
answer.set_index('Userid').to_csv('answer.csv')

In [98]:
X = traindata.drop(['Success'], axis = 1)
y = traindata['Success']

In [99]:
for col in ["Mother's qualification', 'Father's qualification', 'Previous qualification"]:
    X.replace(regex={col: {r"(?i)^no school.*$": 0,
                           r"(?i)^[0-4][a-z]{2} grade.*$": 1,
                           r"(?i)^[5-9]th grade.*$": 2,
                           r"(?i)^1[0-2]th grade.*$": 3,
                           r"(?i)^incomplete bachelor.*$": 4,
                           r"(?i)^bachelor degree.*$": 5,
                           r"(?i)^post-grad.*$": 6,
                           r"(?i)^master degree.*$": 7,
                           r"(?i)^phd.*$": 8
                           }}, inplace=True)
#end for

for col in ["Mother's occupation", "Father´s occupation"]:
    # X.replace
    break

,Marital status,Application mode,Application order,Course,Morning shift participation,Previous qualification,Previous qualification score,Nationality,Mother's qualification,Father's qualification,...,N units approved 1st period,Average grade 1st period,N unscored units 1st period,N units credited 2nd period,N units taken 2nd period,N scored units 2nd period,N units approved 2nd period,Average grade 2nd period,N unscored units 2nd period,Social Popularity
Userid,,,,,,,,,,,,,,,,,,,,,
699,single,0.0,1.0,Agronomy,1.0,3.0,140.0,Portuguese,3.0,2.0,...,5.0,12.600000,0.0,0.0,6.0,11.0,6.0,11.714286,0.0,70.0
456,married,0.0,1.0,Social Service (evening attendance),0.0,3.0,130.0,Portuguese,NaN,NaN,...,6.0,11.166667,0.0,1.0,6.0,15.0,2.0,11.000000,1.0,55.0
647,single,1.0,1.0,Social Service,1.0,3.0,135.0,Portuguese,1.0,3.0,...,5.0,11.200000,0.0,0.0,6.0,15.0,1.0,10.000000,0.0,67.0
383,single,1.0,1.0,Social Service,1.0,3.0,135.0,Portuguese,2.0,2.0,...,3.0,10.333333,0.0,0.0,6.0,12.0,3.0,10.333333,0.0,93.0
1217,single,2.0,4.0,Social Service,1.0,3.0,124.0,Portuguese,2.0,2.0,...,5.0,12.000000,0.0,0.0,6.0,11.0,NaN,12.000000,0.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3462,divorced,0.0,1.0,Nursing,1.0,3.0,130.0,Portuguese,1.0,1.0,...,5.0,13.126000,1.0,1.0,8.0,12.0,5.0,13.126000,1.0,6.0
1157,single,2.0,1.0,Tourism,1.0,3.0,123.0,Portuguese,2.0,1.0,...,3.0,11.666667,0.0,0.0,6.0,15.0,1.0,10.000000,0.0,2.0
227,married,0.0,1.0,Management,1.0,5.0,130.0,Portuguese,3.0,7.0,...,5.0,11.000000,0.0,0.0,5.0,5.0,5.0,13.800000,0.0,12.0


---

In [5]:
import math, sys, numpy as np, pandas as pd
from typing import Literal;
from sklearn import preprocessing;
from sklearn.ensemble import RandomForestClassifier;

def encodeCols(trainData: pd.DataFrame, testData: pd.DataFrame, columns: list[str], /):
  """Encode categorical columns to numerical.

  Args:
      trainData (pd.DataFrame): The train dataframe where conversion is to be performed.
      testData (pd.DataFrame): The test dataframe where conversion is to be performed.
      columns (list[str]): The columns to be converted.
  """;

  for (col) in columns:
    le = preprocessing.LabelEncoder();
    dfCombined = le.fit_transform(pd.concat([trainData[col], testData[col]], axis=0));

    trainLength = trainData[col].__len__();

    # dfCombined.

    trainData[col] = dfCombined[:trainLength];
    testData[col] = dfCombined[trainLength:];
  # end for
# end encodeCols

def importData(trainPath: str = "./data/train.csv", testPath: str = "./data/test.csv") -> tuple[ tuple[pd.DataFrame, pd.Series], pd.DataFrame ]:
  """Imports the training and testing dataset and returns a training feature/target tuple and the testing features.

  Args:
    `trainPath` (`str`): the path (relative or absolute) to the training data csv.
    `testPath` (`str`): the path (relative or absolute) to the testing data csv.

  Returns:
    `tuple[tuple[pd.DataFrame, pd.Series], pd.DataFrame]` - `( (trainX, trainY), testX )`
  """;

  userIdArray: list[Literal["Userid"]] = ["Userid"];

  trainData: pd.DataFrame = pd.read_csv(trainPath, sep=",").drop("Observations", axis=1).drop_duplicates().set_index("Userid");
  # pd.DataFrame.it.drop_duplicates() only removes rows where all values are the same; does not drop if different values but same userId.

  testX: pd.DataFrame = pd.read_csv(testPath, sep=",").drop("Observations", axis=1).set_index(userIdArray);
  if (testX.duplicates().__len__() > 0): raise ValueError("Test data contains duplicate user IDss.");

  trainY: pd.Series = trainData["Success"];
  trainX: pd.Series = trainData.drop(["Success"], axis = 1);

  del trainData, userIdArray;

  return (
    (trainX, trainY),
    testX
  );
#end importData

def main() -> int:
  userIdArray: list[Literal["Userid"]] = ["Userid"];
  trainData: pd.DataFrame = pd.read_csv(path + "project_data/train.csv", sep=",").drop("Observations", axis=1).set_index(userIdArray);
  testData: pd.DataFrame = pd.read_csv(path + "project_data/test.csv", sep=",").drop("Observations", axis=1).set_index(userIdArray);

  del userIdArray;

  # trainData.index = testData.index = ["Userid"];

  # trainData.groupby("Userid");

  # rows without a "registered" value in the dataset are garbage (userId & outcome only, no use for ML algo).
  trainData.drop(trainData[trainData["Registered"] != "Yes"].index, inplace=True);

  trainData = trainData.drop("Registered", axis=1);
  testData = testData.drop("Registered", axis=1);


  encodeCols(trainData, testData, [
    "Marital status",
    "Course",
    "Previous qualification",
    "Nationality",
    "Mother's qualification",
    "Father's qualification",
    "Mother's occupation",
    "Father's occupation"
  ]);

  # drop = [
  #   "Nationality",
  #   "International",
  #   "Special needs",
  #   "Displaced",
  #   "Morning shift participation",
  #   "Previous qualification",
  #   "Marital status",
  #   'Application mode',
  #   'Application order',
  #   'Course',
  #   'Previous qualification score',
  #   "Mother's qualification",
  #   "Father's qualification",
  #   "Mother's occupation",
  #   "Father's occupation",
  #   #'Entry score',
  #   'Debtor',
  #   'Regularized Fees',
  #   'Gender_Male',
  #   'External Funding',
  #   'Age at enrollment',
  #   'N units credited 1st period',
  #   'N units taken 1st period',
  #   'N scored units 1st period',
  #   #'N units approved 1st period',
  #   #'Average grade 1st period',
  #   'N unscored units 1st period',
  #   'N units credited 2nd period',
  #   'N units taken 2nd period',
  #   'N scored units 2nd period',
  #   #'N units approved 2nd period',
  #   #'Average grade 2nd period',
  #   'N unscored units 2nd period',
  #   #'Social Popularity'
  # ];

  # trainData = trainData.drop(drop, axis=1);
  # testData = testData.drop(drop, axis=1);


  trainX: pd.DataFrame = trainData.drop("Success", axis=1);
  trainY: pd.Series = trainData["Success"];

  model: RandomForestClassifier = RandomForestClassifier(800);
  model.fit(trainX, trainY);

  predictions = model.predict(testData);

  for x, y in zip(list(trainX.columns), model.feature_importances_):
    print(f"{x}: {y*100:.2f}");

  outputData = pd.DataFrame([testData.index, predictions]).T;
  outputData.columns = ["Userid", "Success"];

  outputData.to_csv("./answer.csv", index=False);

  return 0;
# end main
main()

Marital status: 0.42
Application mode: 1.93
Application order: 1.69
Course: 3.14
Morning shift participation: 0.38
Previous qualification: 0.76
Previous qualification score: 3.78
Nationality: 0.31
Mother's qualification: 2.20
Father's qualification: 2.35
Mother's occupation: 2.25
Father's occupation: 2.74
Entry score: 4.39
Displaced: 0.99
Special needs: 0.13
Debtor: 1.23
Regularized Fees: 3.84
Gender_Male: 1.21
External Funding: 1.92
Age at enrollment: 4.16
International: 0.28
N units credited 1st period: 0.80
N units taken 1st period: 2.15
N scored units 1st period: 3.67
N units approved 1st period: 9.34
Average grade 1st period: 7.17
N unscored units 1st period: 0.62
N units credited 2nd period: 0.69
N units taken 2nd period: 2.30
N scored units 2nd period: 4.53
N units approved 2nd period: 14.03
Average grade 2nd period: 9.71
N unscored units 2nd period: 0.64
Social Popularity: 4.25


0